In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/drive
Tue Mar  5 13:46:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [ ]:
pip install tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import numpy as np
import sys
import tsfresh as tsf
from tsfresh.utilities.dataframe_functions import impute

In [ ]:

df_outf = pd.read_csv('combined_demand_df_1h.csv')
df_outf = df_outf.drop('time_slot', axis=1)

df_inf = pd.read_csv('combined_supply_df_1h.csv')
df_inf = df_inf.drop('time_slot', axis=1)

if(df_outf.shape[0] != df_inf.shape[0]): # check if dem and inf are same length
    print("outflow and inflow are NOT of same length")
    sys.exit()

In [ ]:
df_outf

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
0,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,7.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,20.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,14.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,5.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,8.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,26.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,19.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,26.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,24.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


In [ ]:
class create_raw_features(object):
    #def __init__(self,):

    def create_lag_features(ts,lags_list):
        lag_ts0 = ts.shift(periods = lags_list[0])
        lag_ts1 = ts.shift(periods = lags_list[1])
        lag_ts2 = ts.shift(periods = lags_list[2])
        lag_ts3 = ts.shift(periods = lags_list[3])
        lag_ts4 = ts.shift(periods = lags_list[4])
        return lag_ts0,lag_ts1,lag_ts2,lag_ts3,lag_ts4

    def create_diff_ts_features(ts,diff_list):
        diff1_p1 = ts.diff(periods=diff_list[0])
        diff1_p2 = ts.diff(periods=diff_list[1])

        diff2_p1 = ts.diff(periods=diff_list[0]).diff()
        return diff1_p1,diff1_p2,diff2_p1

In [ ]:
dt = pd.Timestamp("2021-01-01") + pd.to_timedelta(range(df_inf.shape[0]), "H")
raw_feat_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag_0','inflow_lag_1','inflow_lag_2','inflow_lag_3','inflow_lag_4','outflow_lag_0','outflow_lag_1','outflow_lag_2','outflow_lag_3','outflow_lag_4'])

zone_siginificant_lags= {'4': [  1, 167, 166, 142, 143, 168, 3,  22, 141, 165],
  '12': [ 1, 21, 23, 22, 20,  2, 24,  4,  5, 19],
  '13': [  1,  23,  22,  17,  24,  21,   5,  18, 143, 167],
  '24': [  1,  23,  24,  22,  21, 168, 167, 143, 144,  20],
  '41': [  1,  23,  24,  22, 168, 144, 143, 167,  21,  20],
  '42': [  1,  23,  24, 168,  22,   2, 167, 144,  21, 143],
  '43': [ 1,  4, 25, 24, 23,  3, 21,  7,  6,  2],
  '45': [  1,   2,  23,  22, 167,  21, 168,  19,  20,  24],
  '48':  [ 1,  25,   2,  23,  22,  18, 167, 143,  19,  20],
  '50': [  1,  23,  22,  19,  21,  20, 167,  24, 143,  18],
  '68': [ 1,  2,  3, 19, 23, 25, 18,  4, 20, 22],
  '74': [  1,  23,  22,  24,  21, 168,  20, 167, 143, 144],
  '75': [ 1,  5,  4, 23, 24,  3, 14,  6, 20, 26],
  '79': [  1,   2, 142, 166,  25, 141, 169, 165, 167,  21],
  '87': [  1,  18,  19,  23,   5,  20,  17,   6,  24, 167],
  '88': [ 1,  5,  4,  6, 23,  7, 22, 18, 20,  2],
  '90': [ 1,  3, 19, 25, 22, 18,  4,  2, 21, 23],
  '100':[  1,  23,  22,  24,  25,  21, 167, 143,  20,   2],
  '107': [  1,   2,  25,  23,  19,  18,   3,  22, 167,  20],
  '113': [ 1,  3, 18,  4,  2, 19, 17, 20, 23, 21],
  '114': [  1,   2, 167,  22, 142,  21, 143,  20, 169,  25],
  '116': [  1,  23,  24, 168,  22, 144, 143, 167,  21,  48],
  '120': [  1,  82,  78,  98, 116, 145,   4,   9, 140, 166],
  '125': [  1,  23,  19,  20, 143,  22, 167,  24,  18, 168],
  '127': [  1,   2,  24, 168,   3, 144,  23,  25,  26,  22],
  '128': [ 24,   2, 144,  72,  96, 143,  92,   1,  48, 117],
  '137': [  1,  23,  25,  22,   5,  26,  19,   3, 167,  20],
  '140': [  1,  25,   2,  23,  22,   3,  26,   8,  20, 145],
  '141': [ 1,  2, 23, 25, 19, 17, 18,  5,  4,  3],
  '142': [  1,  23,  24,   2,  22,  19,   5, 168,  18,  25],
  '143': [  1,  23,  22,   2,  24,   5, 167,  19,  21,  20],
  '144': [  1,   3,  23,  22,   2,  20, 167,  25, 166, 142],
  '148': [  1,   2, 166, 141, 165, 142, 167,  21, 169, 140],
  '151': [  1,  23,  22,  24,  21,   5,   6, 168,  11,   2],
  '152': [  1,  23,  24,  22,   2, 168,  21, 144, 167,  11],
  '153': [168,  23,   2,  22,  26,  73,  45,  21,  24,  95],
  '158': [  1, 167,  22, 142,   2,  23, 143,  25, 166,  21],
  '161': [  1,   2,  25,   4,   3,  23,   5, 169,  18,  22],
  '162': [ 1,  2, 25, 23, 22, 17, 18, 19,  3, 21],
  '163': [ 1,  2,  5, 25, 23, 19,  4, 26,  3, 20],
  '164': [ 1,  3, 18, 19,  2,  5,  4, 23, 17, 25],
  '166': [  1,  23,  24,   5,  22,  20, 168,  21,   6,   4],
  '170': [ 1,  2, 25,  3, 19, 18, 20, 23, 22,  5],
  '186': [  1,  23,  25,  22,   2,  24,  21,  20, 168,  19],
  '194': [ 1, 24, 23,  2,  3, 22,  5, 21, 19,  4],
  '202': [  1,  23,  24,   2, 168,   3,  26,  96,  48,  47],
  '209': [  1,  23,  24,  20,  21,  19,  22,   2, 167,   5],
  '211': [  1,  23,   3, 167,  22, 143,  20,  21,  19, 142],
  '224': [  1,  24,  23,   2,  21, 168,  22, 167,  20, 144],
  '229': [ 1,  2, 23, 25, 22, 18, 17, 11, 19, 20],
  '230': [  1,  23,   2,  22,  25,  18,  17,  24, 169, 168],
  '231': [  1,  23,   2,  19,  18,  20,  25,   5, 167, 143],
  '232': [  1,  22,  21,  23,  20, 167, 142, 143, 166, 168],
  '233': [ 1, 23, 22, 24, 19, 25, 20,  5,  3, 18],
  '234': [ 1,  2,  3,  4, 18, 17,  5, 25, 19, 20],
  '236': [  1,   2,  25,   4,  13,  23, 145, 169,   8,  21],
  '237': [  1,   2,  25,   4,  17,   5,  16,  23,   3, 145],
  '238': [ 1, 23,  2, 25,  4, 20, 21, 24, 19, 22],
  '239': [ 1,  2,  4,  5, 23, 19,  3, 20, 17, 18],
  '243': [  1,   2,  24,  23,   3,  22,  25,  21, 168,  20],
  '244': [  1,   2,  24,  23,  22,  16, 168,  10,  21,  20],
  '246': [ 1,  2, 25, 23, 22, 20, 21, 19,  3,  4],
  '249': [  1,   2,  25,  21, 142, 169,  22,  20, 167, 141],
  '261': [ 1,  4,  5,  3, 20, 21, 23,  6, 19, 24],
  '262': [  1,  23,   2,  22,  25,  21,  20, 167, 143,  19],
  '263': [  1,   2,  23,  19,  20,  25,  22,   3, 167,  21]}

clustered_zones = {
    '120': ['4', '120', '249'],
'239': ['68', '75', '88', '113', '163', '164', '170', '194', '202', '234', '238', '239', '261'],
'186': ['13', '107', '137', '144', '158', '186', '209', '230', '231', '243', '263'],
'141': ['12', '43', '90', '141', '162', '229', '233', '246'],
'116': ['41', '42', '116', '127', '211'],
'143': ['45', '48', '50', '100', '125', '143', '166', '244'],
'142': ['142', '151', '161', '236', '262'],
'74': ['24', '74', '152', '224', '232'],
'140': ['87', '140', '153', '237'],
'148': ['79', '114', '128', '148'],
}

clustered_significant_lags = {}

for cluster, zones in clustered_zones.items():
  for zone in zones:
    clustered_significant_lags[zone] = zone_siginificant_lags[cluster][:5]

print(clustered_significant_lags)

sorted_clustered_significant_lags = {k: clustered_significant_lags[k] for k in sorted(clustered_significant_lags, key=lambda x: int(x))}

print(sorted_clustered_significant_lags)

diff_list = [1,2]

{'4': [1, 82, 78, 98, 116], '120': [1, 82, 78, 98, 116], '249': [1, 82, 78, 98, 116], '68': [1, 2, 4, 5, 23], '75': [1, 2, 4, 5, 23], '88': [1, 2, 4, 5, 23], '113': [1, 2, 4, 5, 23], '163': [1, 2, 4, 5, 23], '164': [1, 2, 4, 5, 23], '170': [1, 2, 4, 5, 23], '194': [1, 2, 4, 5, 23], '202': [1, 2, 4, 5, 23], '234': [1, 2, 4, 5, 23], '238': [1, 2, 4, 5, 23], '239': [1, 2, 4, 5, 23], '261': [1, 2, 4, 5, 23], '13': [1, 23, 25, 22, 2], '107': [1, 23, 25, 22, 2], '137': [1, 23, 25, 22, 2], '144': [1, 23, 25, 22, 2], '158': [1, 23, 25, 22, 2], '186': [1, 23, 25, 22, 2], '209': [1, 23, 25, 22, 2], '230': [1, 23, 25, 22, 2], '231': [1, 23, 25, 22, 2], '243': [1, 23, 25, 22, 2], '263': [1, 23, 25, 22, 2], '12': [1, 2, 23, 25, 19], '43': [1, 2, 23, 25, 19], '90': [1, 2, 23, 25, 19], '141': [1, 2, 23, 25, 19], '162': [1, 2, 23, 25, 19], '229': [1, 2, 23, 25, 19], '233': [1, 2, 23, 25, 19], '246': [1, 2, 23, 25, 19], '41': [1, 23, 24, 168, 22], '42': [1, 23, 24, 168, 22], '116': [1, 23, 24, 168, 22]

In [ ]:
"""
stack all the raw feature time series of clusters vertically

tsfresh features are then added to these raw features to enlarge the feature set
"""

for c in df_inf.columns:
    lags_list = sorted_clustered_significant_lags[c]
    temp_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag_0','inflow_lag_1','inflow_lag_2','inflow_lag_3','inflow_lag_4','outflow_lag_0','outflow_lag_1','outflow_lag_2','outflow_lag_3','outflow_lag_4', 'inflow', "inflow_diff1_per1", 'inflow_diff1_per2', 'inflow_diff2_per1'])
    temp_df['time_idx'] = range(df_inf.shape[0])
    temp_df['datetime'] = dt
    temp_df['target'] = df_outf[c]
    temp_df['group'] = np.repeat(int(c),df_inf.shape[0])
    temp_df['inflow_lag_0'],temp_df['inflow_lag_1'],temp_df['inflow_lag_2'],temp_df['inflow_lag_3'],temp_df['inflow_lag_4']= create_raw_features.create_lag_features(df_inf[c],lags_list)
    temp_df['outflow_lag_0'],temp_df['outflow_lag_1'],temp_df['outflow_lag_2'],temp_df['outflow_lag_3'],temp_df['outflow_lag_4']= create_raw_features.create_lag_features(df_outf[c],lags_list)

    temp_df['inflow'] = df_inf[c]
    temp_df['inflow_diff1_per1'],temp_df['inflow_diff1_per2'], temp_df['inflow_diff2_per1'] = create_raw_features.create_diff_ts_features(df_inf[c],diff_list)

    raw_feat_df = pd.concat([raw_feat_df, temp_df],axis=0)

raw_feat_df = raw_feat_df.dropna().reset_index(drop=True)
raw_feat_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,inflow_lag_4,outflow_lag_0,outflow_lag_1,outflow_lag_2,outflow_lag_3,outflow_lag_4,inflow,inflow_diff1_per1,inflow_diff1_per2,inflow_diff2_per1
0,116,2021-01-05 20:00:00,4,3.0,8.0,9.0,15.0,5.0,8.0,0.0,4.0,8.0,0.0,3.0,15.0,7.0,1.0,13.0
1,117,2021-01-05 21:00:00,4,1.0,15.0,5.0,19.0,9.0,13.0,3.0,1.0,2.0,2.0,1.0,7.0,-8.0,-1.0,-15.0
2,118,2021-01-05 22:00:00,4,1.0,7.0,6.0,24.0,4.0,9.0,1.0,1.0,1.0,0.0,2.0,8.0,1.0,-7.0,9.0
3,119,2021-01-05 23:00:00,4,0.0,8.0,5.0,15.0,4.0,5.0,1.0,6.0,2.0,3.0,3.0,4.0,-4.0,-3.0,-5.0
4,120,2021-01-06 00:00:00,4,1.0,4.0,15.0,16.0,5.0,2.0,0.0,8.0,1.0,1.0,0.0,3.0,-1.0,-5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634495,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,194.0,144.0,170.0,142.0,113.0,120.0,82.0,137.0,153.0,-23.0,-17.0,-29.0
1634496,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,166.0,94.0,176.0,126.0,82.0,125.0,77.0,142.0,135.0,-18.0,-41.0,5.0
1634497,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,136.0,62.0,153.0,112.0,77.0,113.0,30.0,126.0,119.0,-16.0,-34.0,2.0
1634498,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,144.0,23.0,135.0,99.0,30.0,82.0,10.0,112.0,119.0,0.0,-16.0,16.0


In [ ]:
"""
Create TSFRESH Time series features

"""

win_lengths = [24] # hyperparameter

# drop all categorical columns

col_list = raw_feat_df.columns

feature_settings = {'fft_coefficient': [{'coeff': 0, 'attr': 'real'}, {'coeff': 1, 'attr': 'real'}],
                    #'mean': None,
                    #'standard_deviation': None,
                    'skewness': None,
                    'kurtosis': None,
                    #'minimum': None,
                    #'maximum': None,
                    #'sum_values': None,
                    #'variance': None,
                    #'absolute_sum_of_changes': None,
                    'count_above_mean': None,
                    'count_below_mean': None,
                    #'longest_strike_above_mean': None,
                    #'longest_strike_below_mean': None,
                    #'number_crossing_m': [{'m': 0}],
                    'autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
                    'partial_autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}]}


for c in col_list[4:]: # select 1 numeric column at a time, leave out target and long lags
    print('time series name: ',c)
    for win in win_lengths:
        num_df = raw_feat_df.loc[:,['datetime','group',c]]
        print('num_df.shape', num_df.shape)
        df_rolled = tsf.utilities.dataframe_functions.roll_time_series(num_df,column_id='group',column_sort='datetime',max_timeshift=win)
        df_rolled.drop(columns=['group'], inplace=True)
        df_features = tsf.extract_features(df_rolled, column_id='id', column_sort='datetime',
                                            default_fc_parameters=feature_settings)
        del df_rolled # free-up the RAM

        # Remove any missing values
        imputed_features = impute(df_features)
        del df_features # free-up the RAM

        # Select a subset of relevant features using tsfresh's built-in feature selection function
        raw_feat_df.index= range(raw_feat_df.shape[0])
        imputed_features.index = range(raw_feat_df.shape[0])
        selected_features = tsf.select_features(imputed_features, raw_feat_df['target'])
        del imputed_features # free-up the RAM

        # Print the names of the selected features
        print('selected_features.columns',selected_features.columns)

        print('selected_features.columns: ',selected_features.columns)
        print('selected_features.shape: ',selected_features.shape)

        print('all_stns_df.columns: ',raw_feat_df.columns)
        print('all_stns_df.shape: ',raw_feat_df.shape)

        raw_feat_df = pd.concat([raw_feat_df,selected_features],axis=1)

        del num_df # free-up the RAM
        del selected_features # free-up the RAM

raw_feat_df.shape

time series name:  inflow_lag_4
num_df.shape (1634500, 3)


Feature Extraction: 100%|██████████| 20/20 [14:28<00:00, 43.44s/it]


selected_features.columns Index(['inflow_lag_4__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_4__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_4__skewness', 'inflow_lag_4__kurtosis',
       'inflow_lag_4__count_above_mean', 'inflow_lag_4__count_below_mean',
       'inflow_lag_4__autocorrelation__lag_1',
       'inflow_lag_4__autocorrelation__lag_2',
       'inflow_lag_4__autocorrelation__lag_3',
       'inflow_lag_4__partial_autocorrelation__lag_1',
       'inflow_lag_4__partial_autocorrelation__lag_2',
       'inflow_lag_4__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_lag_4__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_4__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_4__skewness', 'inflow_lag_4__kurtosis',
       'inflow_lag_4__count_above_mean', 'inflow_lag_4__count_below_mean',
       'inflow_lag_4__autocorrelation__lag_1',
       'inflow_lag_4__autocorrelation__lag_2',
       

Feature Extraction: 100%|██████████| 20/20 [14:22<00:00, 43.11s/it]


selected_features.columns Index(['outflow_lag_0__fft_coefficient__attr_"real"__coeff_0',
       'outflow_lag_0__fft_coefficient__attr_"real"__coeff_1',
       'outflow_lag_0__skewness', 'outflow_lag_0__kurtosis',
       'outflow_lag_0__count_above_mean', 'outflow_lag_0__count_below_mean',
       'outflow_lag_0__autocorrelation__lag_1',
       'outflow_lag_0__autocorrelation__lag_2',
       'outflow_lag_0__autocorrelation__lag_3',
       'outflow_lag_0__partial_autocorrelation__lag_1',
       'outflow_lag_0__partial_autocorrelation__lag_2',
       'outflow_lag_0__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['outflow_lag_0__fft_coefficient__attr_"real"__coeff_0',
       'outflow_lag_0__fft_coefficient__attr_"real"__coeff_1',
       'outflow_lag_0__skewness', 'outflow_lag_0__kurtosis',
       'outflow_lag_0__count_above_mean', 'outflow_lag_0__count_below_mean',
       'outflow_lag_0__autocorrelation__lag_1',
       'outflow_lag_0__autocorrelat

...

Feature Extraction: 100%|██████████| 20/20 [14:28<00:00, 43.43s/it]


selected_features.columns Index(['inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per2__skewness', 'inflow_diff1_per2__kurtosis',
       'inflow_diff1_per2__count_above_mean',
       'inflow_diff1_per2__count_below_mean',
       'inflow_diff1_per2__autocorrelation__lag_1',
       'inflow_diff1_per2__autocorrelation__lag_2',
       'inflow_diff1_per2__autocorrelation__lag_3',
       'inflow_diff1_per2__partial_autocorrelation__lag_1',
       'inflow_diff1_per2__partial_autocorrelation__lag_2',
       'inflow_diff1_per2__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per2__skewness', 'inflow_diff1_per2__kurtosis',
       'inflow_diff1_per2__count_above_mean',
       'inflow_diff1_per2__count_below_mean

Feature Extraction: 100%|██████████| 20/20 [14:41<00:00, 44.09s/it]


selected_features.columns Index(['inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff2_per1__count_above_mean',
       'inflow_diff2_per1__autocorrelation__lag_1',
       'inflow_diff2_per1__autocorrelation__lag_2',
       'inflow_diff2_per1__partial_autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__lag_2',
       'inflow_diff2_per1__partial_autocorrelation__lag_3',
       'inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff2_per1__autocorrelation__lag_3',
       'inflow_diff2_per1__count_below_mean', 'inflow_diff2_per1__kurtosis'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff2_per1__count_above_mean',
       'inflow_diff2_per1__autocorrelation__lag_1',
       'inflow_diff2_per1__autocorrelation__lag_2',
       'inflow_diff2_per1__partial_autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__l

(1634500, 137)

In [ ]:
for c in raw_feat_df.columns:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
inflow_lag_4
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
outflow_lag_4
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_4__fft_coefficient__attr_"real"__coeff_0
inflow_lag_4__fft_coefficient__attr_"real"__coeff_1
inflow_lag_4__skewness
inflow_lag_4__kurtosis
inflow_lag_4__count_above_mean
inflow_lag_4__count_below_mean
inflow_lag_4__autocorrelation__lag_1
inflow_lag_4__autocorrelation__lag_2
inflow_lag_4__autocorrelation__lag_3
inflow_lag_4__partial_autocorrelation__lag_1
inflow_lag_4__partial_autocorrelation__lag_2
inflow_lag_4__partial_autocorrelation__lag_3
outflow_lag_0__fft_coefficient__attr_"real"__coeff_0
outflow_lag_0__fft_coefficient__attr_"real"__coeff_1
outflow_lag_0__skewness
outflow_lag_0__kurtosis
outflow_lag_0__count_above_mean
outflow_lag_0__count_below_mean
outflow_lag_0__autocorrelation__lag_1
outflow_lag_0__autocorrelation__lag_2
outflow_lag_0__autoco

In [ ]:
raw_feat_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,inflow_lag_4,outflow_lag_0,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,116,2021-01-05 20:00:00,4,3.0,8.0,9.0,15.0,5.0,8.0,0.0,...,0.0,-0.517260,0.028563,-0.517251,-0.362922,-0.207774,13.0,-0.000421,0.0,0.574562
1,117,2021-01-05 21:00:00,4,1.0,15.0,5.0,19.0,9.0,13.0,3.0,...,1.0,-1.000000,0.028563,-0.517251,-0.362922,-0.207774,-2.0,-0.000421,1.0,0.574562
2,118,2021-01-05 22:00:00,4,1.0,7.0,6.0,24.0,4.0,9.0,1.0,...,2.0,-0.982558,0.465116,-0.517251,-0.362922,-0.207774,7.0,-0.000421,1.0,0.574562
3,119,2021-01-05 23:00:00,4,0.0,8.0,5.0,15.0,4.0,5.0,1.0,...,2.0,-0.994656,0.767535,-0.994656,-0.362922,-0.207774,2.0,-0.551102,2.0,-2.865193
4,120,2021-01-06 00:00:00,4,1.0,4.0,15.0,16.0,5.0,2.0,0.0,...,3.0,-0.942460,0.687831,-0.942460,-0.362922,-0.207774,5.0,-0.515873,2.0,-0.781557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634495,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,194.0,144.0,170.0,142.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1634496,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,166.0,94.0,176.0,126.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1634497,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,136.0,62.0,153.0,112.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1634498,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,144.0,23.0,135.0,99.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705


In [ ]:
pip install pyarrow

In [ ]:
raw_feat_df.to_parquet('combined_dd_ss_raw_feat_data_significant_lags_cluster.parquet')

In [ ]:
pd.read_parquet('combined_dd_ss_raw_feat_data_significant_lags_cluster.parquet', engine='pyarrow')

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,inflow_lag_4,outflow_lag_0,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,116,2021-01-05 20:00:00,4,3.0,8.0,9.0,15.0,5.0,8.0,0.0,...,0.0,-0.517260,0.028563,-0.517251,-0.362922,-0.207774,13.0,-0.000421,0.0,0.574562
1,117,2021-01-05 21:00:00,4,1.0,15.0,5.0,19.0,9.0,13.0,3.0,...,1.0,-1.000000,0.028563,-0.517251,-0.362922,-0.207774,-2.0,-0.000421,1.0,0.574562
2,118,2021-01-05 22:00:00,4,1.0,7.0,6.0,24.0,4.0,9.0,1.0,...,2.0,-0.982558,0.465116,-0.517251,-0.362922,-0.207774,7.0,-0.000421,1.0,0.574562
3,119,2021-01-05 23:00:00,4,0.0,8.0,5.0,15.0,4.0,5.0,1.0,...,2.0,-0.994656,0.767535,-0.994656,-0.362922,-0.207774,2.0,-0.551102,2.0,-2.865193
4,120,2021-01-06 00:00:00,4,1.0,4.0,15.0,16.0,5.0,2.0,0.0,...,3.0,-0.942460,0.687831,-0.942460,-0.362922,-0.207774,5.0,-0.515873,2.0,-0.781557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634495,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,194.0,144.0,170.0,142.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1634496,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,166.0,94.0,176.0,126.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1634497,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,136.0,62.0,153.0,112.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1634498,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,144.0,23.0,135.0,99.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705
